<a href="https://colab.research.google.com/github/Rochitha-prabhuram/Digital-EHR/blob/main/NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install pdfplumber pymupdf pytesseract easyocr layoutparser torch pillow opencv-python


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 497.9 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.8/67.8 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 3.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 116.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 92.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 102.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.2/19.2 MB 92.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 102.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 180.7/180.7 kB 17.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 978.2/978.2 kB 56.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.6/300.6 kB 20.3 MB/s eta 0:00:00
  Cre

In [2]:
from google.colab import files
uploaded = files.upload()  # choose your PDF
pdf_path = list(uploaded.keys())[0]

Saving DocScanner 7 Jan 2026 9-52 pm.pdf to DocScanner 7 Jan 2026 9-52 pm.pdf


In [3]:
def extract_text_digital(pdf_path):
    text_data = []
    with pdfplumber.open(pdf_path) as pdf:
        for i, page in enumerate(pdf.pages):
            text = page.extract_text()
            if text:
                text_data.append({
                    "page": i + 1,
                    "text": text.strip()
                })
    return text_data


In [4]:
import fitz  # PyMuPDF
import os

def pdf_to_images(pdf_path, output_dir="pages"):
    os.makedirs(output_dir, exist_ok=True)
    doc = fitz.open(pdf_path)
    image_paths = []

    for i, page in enumerate(doc):
        pix = page.get_pixmap(dpi=300)
        img_path = f"{output_dir}/page_{i+1}.png"
        pix.save(img_path)
        image_paths.append(img_path)

    return image_paths


In [5]:
import easyocr

reader = easyocr.Reader(['en'], gpu=True)

def ocr_images(image_paths):
    ocr_results = []

    for idx, img in enumerate(image_paths):
        result = reader.readtext(img, detail=0, paragraph=True)
        ocr_results.append({
            "page": idx + 1,
            "text": "\n".join(result)
        })

    return ocr_results


Progress: |██████████████████████████████████████████████████| 100.0% Complete

Progress: |██████████████████████████████████████████████████| 100.0% Complete

In [6]:
def extract_text_from_pdf(pdf_path):
    if is_scanned_pdf(pdf_path):
        images = pdf_to_images(pdf_path)
        text = ocr_images(images)
        source = "scanned_pdf_ocr"
    else:
        text = extract_text_digital(pdf_path)
        source = "digital_pdf"

    return {
        "file": pdf_path,
        "source_type": source,
        "pages": text
    }
